<a href="https://colab.research.google.com/github/DietzscheNostoevsky/Learning_Pytorch/blob/main/07_Pytorch_Experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Experiment Tracking

# Milestone Project 1: FoodVision Mini Experiment Tracking.

# 0. Setup

In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
torch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118


In [2]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 3808, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (268/268), done.
remote: Total 3808 (delta 234), reused 354 (delta 176), pack-reused 3357
Receiving objects: 100% (3808/3808), 650.23 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (2190/2190), done.
Updating files: 100% (248/248), done.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

# 1 Getting Data

In [6]:
import os 
import zipfile 
from pathlib import Path
import requests 
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

## Smaller Dataset

In [5]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str, 
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.
    
    Returns:
        pathlib.Path to downloaded data.
    
    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)
        
        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...") 
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)
    
    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] Did not find data/pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


PosixPath('data/pizza_steak_sushi')

In [7]:
walk_through_dir(image_path)

There are 2 directories and 0 images in 'data/pizza_steak_sushi'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/test'.
There are 0 directories and 19 images in 'data/pizza_steak_sushi/test/steak'.
There are 0 directories and 25 images in 'data/pizza_steak_sushi/test/pizza'.
There are 0 directories and 31 images in 'data/pizza_steak_sushi/test/sushi'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/train'.
There are 0 directories and 75 images in 'data/pizza_steak_sushi/train/steak'.
There are 0 directories and 78 images in 'data/pizza_steak_sushi/train/pizza'.
There are 0 directories and 72 images in 'data/pizza_steak_sushi/train/sushi'.


## Larger Dataset

### Getting larger dataset from Drive

In [8]:
# Mount the GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
data_path = Path("data/") # The trailing forward slash (/) in the string 
                          # is used to indicate that it represents a 
                          # directory rather than a specific file. 
                          # It's a common convention to include the trailing slash 
                          # in directory paths to differentiate them from file paths.

image_path_full = data_path / "pizza_steak_sushi_full"

if image_path.is_dir():
    print(f"{image_path_full} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path_full.mkdir(parents=True, exist_ok=True)
zip_data = "/content/drive/Othercomputers/My MacBook Air/GitHub/-Machine_Learning/Learning_Pytorch/pizza_steak_sushi_100_percent.zip"

with zipfile.ZipFile(zip_data, "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...") 
        zip_ref.extractall(image_path_full)
walk_through_dir(image_path_full)

data/pizza_steak_sushi_full directory exists.
Unzipping pizza, steak, sushi data...
There are 2 directories and 0 images in 'data/pizza_steak_sushi_full'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi_full/test'.
There are 0 directories and 250 images in 'data/pizza_steak_sushi_full/test/steak'.
There are 0 directories and 250 images in 'data/pizza_steak_sushi_full/test/pizza'.
There are 0 directories and 250 images in 'data/pizza_steak_sushi_full/test/sushi'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi_full/train'.
There are 0 directories and 750 images in 'data/pizza_steak_sushi_full/train/steak'.
There are 0 directories and 750 images in 'data/pizza_steak_sushi_full/train/pizza'.
There are 0 directories and 750 images in 'data/pizza_steak_sushi_full/train/sushi'.


# 3. Creating Datasets and Dataloaders

In [16]:
# Setup Dirs 

#----------------------------------
# small dataset
train_dir = image_path / "train"
test_dir = image_path / "test"
#----------------------------------
#full dataset
#train_dir = image_path_full / "train"
#test_dir = image_path_full / "test"

## 2.1 Create DataLoaders using manually created transforms

In [17]:
print("Train dir : ", train_dir)
print("Test dir :", test_dir)

# Setup ImageNet normalization levels (turns all images into similar distribution as ImageNet)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Create transform pipeline manually

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])           
print(f"Manually created transforms: {manual_transforms}")

# Create data loaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms, # use manually created transforms
    batch_size=32
)
print(F"Training Images : {len(train_dataloader.dataset)}")
print(F"Testing Images : {len(test_dataloader.dataset)}")

print("train_dataloader: ", train_dataloader)
print("test_dataloader: ", test_dataloader)

print(F"Class names : {class_names}")

Train dir :  data/pizza_steak_sushi/train
Test dir : data/pizza_steak_sushi/test
Manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
Training Images : 225
Testing Images : 75
train_dataloader:  <torch.utils.data.dataloader.DataLoader object at 0x7fe5fa5a2320>
test_dataloader:  <torch.utils.data.dataloader.DataLoader object at 0x7fe5fa5a1ed0>
Class names : ['pizza', 'steak', 'sushi']


## 2.2 Creating DataLoaders using automatically created transforms

In [18]:
print("Train dir : ", train_dir)
print("Test dir :", test_dir, "\n")

# Setup pretrained weights (plenty of these available in torchvision.models)
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# Get transforms from weights (these are the transforms that were used to obtain the weights)
automatic_transforms = weights.transforms() 
print(f"Automatically created transforms: {automatic_transforms}")

# Create data loaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=automatic_transforms, # use automatic created transforms
    batch_size=32
)

print(F"Training Images : {len(train_dataloader.dataset)}")
print(F"Testing Images : {len(test_dataloader.dataset)}")

print("train_dataloader: ", train_dataloader)
print("test_dataloader: ", test_dataloader)

print(F"Class names : {class_names}")

Train dir :  data/pizza_steak_sushi/train
Test dir : data/pizza_steak_sushi/test 

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)
Training Images : 225
Testing Images : 75
train_dataloader:  <torch.utils.data.dataloader.DataLoader object at 0x7fe5fa5a3dc0>
test_dataloader:  <torch.utils.data.dataloader.DataLoader object at 0x7fe5fa5a29e0>
Class names : ['pizza', 'steak', 'sushi']


In the given transformation, the `crop_size` and `resize_size` parameters serve different purposes.

1. `crop_size`: This parameter determines the size of the image after cropping. It specifies the dimensions (width and height) of the cropped image. During training or evaluation, the input image is typically cropped to a fixed size to ensure consistent input dimensions for the model. In this case, the `crop_size` is set to `[224]`, which means the cropped image will have a width and height of 224 pixels.

2. `resize_size`: This parameter determines the size of the image after resizing. It specifies the dimensions (width and height) to which the image should be resized. Resizing is commonly performed to standardize the input image size for the model. In this case, the `resize_size` is set to `[256]`, indicating that the image will be resized to a width and height of 256 pixels.

The purpose of using both `crop_size` and `resize_size` is to apply a two-step transformation to the input images. First, the image is resized to a larger size (`resize_size`). Then, a central crop of the specified size (`crop_size`) is extracted from the resized image.

By applying this two-step transformation, the model receives inputs that have been both resized and cropped, ensuring that the model receives consistent input sizes and capturing different scales and perspectives of the original image.

The other parameters (`mean`, `std`, `interpolation`) specify additional image transformation details such as mean normalization, standard deviation normalization, and the interpolation method used during resizing.

# 3. Getting a pretrained model, freezing the base layers and changing the classifier head


In [20]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # best weights available 

model = torchvision.models.efficientnet_b0(weights=weights).to(device)

#model

We'll freeze the base layers of the model (we'll use these to extract features from our input images) and we'll change the classifier head (output layer) to suit the number of classes we're working with (we've got 3 classes: pizza, steak, sushi).

In [21]:
# Freeznig the base layers 
for param in model.features.parameters():
    param.requires_grad = False 

# Since we're creating a new layer with random weights (torch.nn.Linear), 
# let's set the seeds
set_seeds()
model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names),
              bias=True)
).to(device)


In [23]:
from torchinfo import summary

# # Get a summary of the model (uncomment for full output)
summary(model, 
         input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
         verbose=0,
         col_names=["input_size", "output_size", "num_params", "trainable"],
         col_width=20,
         row_settings=["var_names"]
 )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 